In [30]:
import numpy as np
import matplotlib.pyplot as plt
import warnings
from warnings import warn
%load_ext autoreload
%autoreload 2

from BenamouBrenier import TransportProblem
from transport import gaussian_transport, gaussian_discreatization

rng = np.random.default_rng(4321)

In [43]:
a=(1,2,3)
2*a

(1, 2, 3, 1, 2, 3)

In [34]:
N=5
mesh= np.random.randn(N,2)
mu = np.sum(mesh,axis=1)**2
nu = 2*np.sum(mesh,axis=1)
T = 10

prob1 = TransportProblem(mesh,mu,nu,T)
print(prob1)

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 3 dimension(s) and the array at index 1 has 4 dimension(s)